In [6]:
%pip install xlrd
# Loading the Dataset & Inspect Structure
import pandas as pd

# Load the Excel file (use read_excel instead of read_csv)
df = pd.read_excel('C:/Users/akhil/Downloads/whc-sites-2019 - Copy.xls')

print("========== [MODULE 1: Dataset Overview] ==========\n")
print(" Dataset Loaded Successfully!")
print(f" Total Records   : {df.shape[0]}")
print(f" Total Features  : {df.shape[1]}\n")

print("Column Names:")
print("-" * 50)
for col in df.columns:
    print(f"• {col}")
print("-" * 50)

print("\nSample Records (Top 5 Rows):")
print(df.head().to_markdown(index=False))


Note: you may need to restart the kernel to use updated packages.
========== [MODULE 1: Dataset Overview] ==========

 Dataset Loaded Successfully!
 Total Records   : 1121
 Total Features  : 22

Column Names:
--------------------------------------------------
• category
• states_name_en
• region_en
• unique_number
• id_no
• rev_bis
• name_en
• short_description_en
• justification_en
• date_inscribed
• secondary_dates
• danger
• date_end
• danger_list
• longitude
• latitude
• area_hectares
• criteria_txt
• category_short
• iso_code
• udnp_code
• transboundary
--------------------------------------------------

Sample Records (Top 5 Rows):
| category   | states_name_en   | region_en                |   unique_number |   id_no | rev_bis   | name_en                                                             | short_description_en                                                                                                                                                                   

In [7]:
# Removing Duplicates
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]

print("\n========== [MODULE 2: Duplicate Removal] ==========")
print(f" Rows Before Duplicate Removal : {before}")
print(f" Duplicates Removed            : {before - after}")
print(f" Rows After Duplicate Removal  : {after}")



========== [MODULE 2: Duplicate Removal] ==========
 Rows Before Duplicate Removal : 1121
 Duplicates Removed            : 0
 Rows After Duplicate Removal  : 1121


In [8]:
# Checking Missing Values
missing = df.isnull().sum()
missing = missing[missing > 0]

print("\n========== [MODULE 3: Missing Value Check] ==========")
if not missing.empty:
    print("❗ Missing Values Found:\n")
    print(missing.to_frame(name='Missing Count').to_markdown())
else:
    print("✅ No Missing Values Detected!")


========== [MODULE 3: Missing Value Check] ==========
❗ Missing Values Found:

|                  |   Missing Count |
|:-----------------|----------------:|
| rev_bis          |             854 |
| justification_en |             787 |
| secondary_dates  |            1034 |
| date_end         |            1082 |
| danger_list      |            1033 |
| area_hectares    |              20 |
| iso_code         |               1 |
| udnp_code        |               1 |


In [12]:
# ======== [MODULE 0: Setup] ========
import pandas as pd
import os
import subprocess

# Optional: Install xlrd if not already installed
try:
    import xlrd
except ImportError:
    subprocess.check_call(["pip", "install", "xlrd"])
    import xlrd

# ======== [MODULE 1: Load Dataset] ========
file_path = "C:/Users/akhil/Downloads/whc-sites-2019 - Copy.xls"
df = pd.read_excel(file_path)

print("\n========== [MODULE 1: Dataset Overview] ==========")
print(f"✅ Dataset Loaded Successfully!")
print(f"🔢 Total Records: {df.shape[0]}")
print(f"📌 Total Features: {df.shape[1]}")
print("\n📂 Columns List:")
print(df.columns.to_list())

# ======== [MODULE 2: Detect Numeric Columns] ========
numeric_cols = df.select_dtypes(include='number').columns.tolist()
print("\n========== [MODULE 2: Numeric Column Summary] ==========")
print(f"🔍 Numeric Columns Found: {numeric_cols}")
print(df[numeric_cols].describe().T)

# ======== [MODULE 3: Outlier Detection & Removal] ========
print("\n========== [MODULE 3: Outlier Detection] ==========")

# Example outlier rules (customize based on actual data inspection)
outlier_conditions = (
    (df['date_inscribed'] > 2025) |
    (df['longitude'] < -180) | (df['longitude'] > 180) |
    (df['latitude'] < -90) | (df['latitude'] > 90)
)

print(f"🧹 Outliers to be removed: {outlier_conditions.sum()} rows")

# Remove outliers
df_clean = df[~outlier_conditions]

print(f"✅ Remaining Rows After Cleaning: {df_clean.shape[0]}")

# ======== [MODULE 4: Save Cleaned Data (Optional)] ========
# df_clean.to_excel("cleaned_whc_sites.xlsx", index=False)



========== [MODULE 1: Dataset Overview] ==========
✅ Dataset Loaded Successfully!
🔢 Total Records: 1121
📌 Total Features: 22

📂 Columns List:
['category', 'states_name_en', 'region_en', 'unique_number', 'id_no', 'rev_bis', 'name_en', 'short_description_en', 'justification_en', 'date_inscribed', 'secondary_dates', 'danger', 'date_end', 'danger_list', 'longitude', 'latitude', 'area_hectares', 'criteria_txt', 'category_short', 'iso_code', 'udnp_code', 'transboundary']

========== [MODULE 2: Numeric Column Summary] ==========
🔍 Numeric Columns Found: ['unique_number', 'id_no', 'date_inscribed', 'danger', 'date_end', 'longitude', 'latitude', 'area_hectares', 'transboundary']
                 count           mean           std          min          25%  \
unique_number   1121.0    1170.794826  6.666083e+02     4.000000   638.000000   
id_no           1121.0     803.338983  4.564630e+02     1.000000   405.000000   
date_inscribed  1121.0    1997.819804  1.165530e+01  1978.000000  1987.000000

In [13]:
# MODULE 3: Detect Outliers (based on valid numeric columns)
print("\n========== [MODULE 3: Outlier Detection] ==========")

# Rename fields to match dataset
outlier_conditions = (
    (df['date_inscribed'] > 2025) |
    (df['latitude'] < -90) | (df['latitude'] > 90) |
    (df['longitude'] < -180) | (df['longitude'] > 180)
)

print(f"❌ Outliers Detected: {outlier_conditions.sum()} rows")

# Remove outliers
df_clean = df[~outlier_conditions]

print(f"✅ Remaining Rows After Cleaning: {df_clean.shape[0]}")



========== [MODULE 3: Outlier Detection] ==========
❌ Outliers Detected: 0 rows
✅ Remaining Rows After Cleaning: 1121


In [15]:
# ======== [MODULE 5: Feature Engineering] ========
# 1. Age of Site (based on inscription year)
df['Age of Site (Years)'] = 2025 - df['date_inscribed']

# 2. Age Group
df['Age Group'] = pd.cut(df['Age of Site (Years)'],
                         bins=[0, 25, 50, 100, 200, 1000],
                         labels=['0-25', '26-50', '51-100', '101-200', '200+'])

# 3. Hemisphere based on latitude
df['Hemisphere'] = df['latitude'].apply(lambda x: 'Northern' if x >= 0 else 'Southern')

print("\n========== [MODULE 5: Feature Engineering] ==========")
print("🆕 New Features Added:")
print("   • Age of Site (Years)")
print("   • Age Group")
print("   • Hemisphere\n")

print("📊 Sample of Engineered Features:\n")
print(df[['name_en', 'date_inscribed', 'Age of Site (Years)',
          'Age Group', 'latitude', 'Hemisphere']].head().to_markdown(index=False))



========== [MODULE 5: Feature Engineering] ==========
🆕 New Features Added:
   • Age of Site (Years)
   • Age Group
   • Hemisphere

📊 Sample of Engineered Features:

| name_en                                                             |   date_inscribed |   Age of Site (Years) | Age Group   |   latitude | Hemisphere   |
|:--------------------------------------------------------------------|-----------------:|----------------------:|:------------|-----------:|:-------------|
| Cultural Landscape and Archaeological Remains of the Bamiyan Valley |             2003 |                    22 | 0-25        |    34.8469 | Northern     |
| Minaret and Archaeological Remains of Jam                           |             2002 |                    23 | 0-25        |    34.3964 | Northern     |
| Historic Centres of Berat and Gjirokastra                           |             2005 |                    20 | 0-25        |    40.0694 | Northern     |
| Butrint                                      

In [16]:
# Saving Cleaned Data
df.to_csv("Cleaned_Dataset.csv", index=False)

print("\n========== [MODULE 6: Save Cleaned Dataset] ==========")
print("💾 Cleaned dataset successfully saved as Processed_Dataset.csv")


========== [MODULE 6: Save Cleaned Dataset] ==========
💾 Cleaned dataset successfully saved as Processed_Dataset.csv
